In [ ]:
#
# --- 1. Installation and Setup ---
#
!pip install gdown
!pip install pyyaml

In [ ]:
# Clone your git repository or upload the files
# For example, if your code is on GitHub (in a public repo)
# !git clone https://github.com/your-username/weather_forecast.git
# %cd weather_forecast

# Install the package
!pip install .

In [ ]:
#
# --- Import some libraries needed for the Configuration step ---
#
from google.colab import drive
drive.mount('/content/drive')

import yaml
from pathlib import Path

In [ ]:
#
# --- 2. Configuration ---
#
config_path = "config.yaml"

with open(config_path, "r") as f:
    config = yaml.safe_load(f)

# Update paths for Google Drive
gdrive_path = Path(config['gdrive_path'])
config['data_path'] = str(gdrive_path / 'data')
config['models_path'] = str(gdrive_path / 'saved_models')

# Create directories on Google Drive if they don't exist
Path(config['data_path']).mkdir(parents=True, exist_ok=True)
Path(config['models_path']).mkdir(parents=True, exist_ok=True)

with open(config_path, "w") as f:
    yaml.dump(config, f)

In [ ]:
#
# --- 3. Run Training ---
#
!weather-forecast-train


In [ ]:
#
# --- 4. Visualize Predictions ---
#
from weather_forecast.train import models
from weather_forecast.utils.window_generator import WindowGenerator
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the trained model
model_class = models.get(config['model_name'])
model = model_class.load(config['models_path'])

# Load and prepare data for prediction
file_list = list(Path(config['data_path']).glob("*.csv"))
raw_df = data_handler.load_data(file_list)
cleaned_df = data_handler.clean_data(raw_df.copy())
station_df = cleaned_df[cleaned_df['station_id'] == 'station_1'].copy()
station_df.drop('station_id', axis=1, inplace=True)
test_df = station_df[int(len(station_df) * 0.9):]
test_mean = test_df.mean()
test_std = test_df.std()
test_df = (test_df - test_mean) / test_std
num_features = station_df.shape[1]

# Create a window generator for the test data
test_window = WindowGenerator(
    input_width=config['input_width'],
    label_width=config['out_steps'],
    shift=config['out_steps'],
    train_df=pd.DataFrame(), # Not needed for prediction
    val_df=pd.DataFrame(),   # Not needed for prediction
    test_df=test_df,
    label_columns=[config['label_column']]
)

# Plot predictions
test_window.plot(model.model)
